# Differences in MT and SP in parasha #2 Noach (Genesis 6:9-11:32)

## Table of Content <a class="anchor" id="TOC"></a> (ToC)

* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Load Text-Fabric app and data</a>
* <a href="#bullet3">3 - Compare surface texts of SP and MT</a>
* <a href="#bullet4">4 - Compare texts using minimum Levenshtein distance</a>
* <a href="#bullet5">5 - Comparison of spelling of proper nouns between SP and MT</a>
* <a href="#bullet6">6 - References and acknowledgement</a>
* <a href="#bullet7">7 - Required libraries</a>
* <a href="#bullet8">8 - Notebook version details</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

The Samaritan Pentateuch (SP) is a version of the Torah preserved by the Samaritan community, differing from the Masoretic Text (MT) in several aspects, including language, orthography, and occasionally theological emphasis. This notebook compares the text of the Masoretic Text, based on the BHSA dataset in Text-Fabric, with the Samaritan Pentateuch, also available as a Text-Fabric dataset.<a href="#ref1"><sup>1</sup></a>

In this analysis, we focus on comparing the text of the verses in a specific parasha, highlighting differences in wording and orthography. Additionally, special attention is given to spelling variations of proper nouns between the two traditions. This notebook draws inspiration from the notebook provided by Martijn Naaijer<a href="#ref2"><sup>2</sup></a> and aims to explore the textual nuances between these two important versions of the Torah.

# 2 - Load Text-Fabric app and data <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

The following code will load the Text-Fabric version of the [Samaritan Pentatuch](https://github.com/DT-UCPH/sp), the [Biblia Hebraica Stuttgartensia (Amstelodamensis)](https://etcbc.github.io/bhsa/) together with the additonal parasha related features from [tonyjurg/BHSaddons](https://github.com/tonyjurg/BHSaddons).

In [1]:
from tf.app import use

# Load the SP data, and rename the node features class F,
# the locality class L and the text class T, 
# then they cannot be overwritten while loading the MT.
SP = use('DT-UCPH/sp', version='3.4')
Fsp, Lsp, Tsp = SP.api.F, SP.api.L, SP.api.T

# Do the same for the MT dataset (BHSA) together with BHSaddons 
MT = use('etcbc/bhsa', version='2021',mod="tonyjurg/BHSaddons/tf/")
Fmt, Lmt, Tmt = MT.api.F, MT.api.L, MT.api.T

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,5,79878.40,100
chapter,187,2135.79,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399392,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


# 3 - Compare surface texts of SP and MT <a class="anchor" id="bullet3"></a>
##### [Back to ToC](#TOC)

In this section, we compare the surface texts of the Samaritan Pentateuch (SP) and the Masoretic Text (MT) at the verse level. By analyzing the wording and structure of these texts, we aim to identify variations.

In [2]:
# find all word nodes for this parasha (we can either use the transliterated name or the sequence number)
parashaQuery = '''
verse parashanum=2
'''
parashaResults = MT.search(parashaQuery)

  0.02s 153 results


In [3]:
# Extract book, chapter, and verse information
bookChapterVerseList = [
    Tmt.sectionFromNode(verse[0]) for verse in parashaResults
]

# Store parashname, start and end verse for future use
startNode=parashaResults[0][0]
endNode=parashaResults[-1][0]
parashaNameHebrew=Fmt.parashahebr.v(startNode)
parashaNameEnglish=Fmt.parashatrans.v(startNode)
bookStart,chapterStart,startVerse=Tmt.sectionFromNode(startNode)
parashaStart=f'{bookStart} {chapterStart}:{startVerse}'
bookEnd,chapterEnd,startEnd=Tmt.sectionFromNode(endNode)
parashaEnd=f'{chapterEnd}:{startEnd}'

In [4]:
# Function to reconstruct verses
def reconstructVerses(F, L, T, textFeature, inputList):
    """Reconstruct text for each verse."""
    verseTexts = {}
    for verseName in inputList:
        verseText = ''
        verseNode = T.nodeFromSection(verseName)
        wordNodes = L.d(verseNode, 'word')
        for wordNode in wordNodes:
            wordText = eval(f'F.{textFeature}.v(wordNode)')
            trailer = F.trailer.v(wordNode)
            if wordText:
                verseText += wordText + (trailer if trailer else ' ')
        verseTexts[verseName] = verseText.strip()
    return verseTexts
    
SPverses = reconstructVerses(Fsp, Lsp, Tsp, 'g_cons', bookChapterVerseList)
MTverses = reconstructVerses(Fmt, Lmt, Tmt, 'g_cons', bookChapterVerseList)

In [5]:
from difflib import SequenceMatcher

def highlightMatches(baseText, comparisonText):
    matcher = SequenceMatcher(None, baseText, comparisonText)
    highlightedComparisonText = "" 
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == "equal":  # Identical parts
            highlightedComparisonText += comparisonText[j1:j2]
        else:  # Non-matching parts
            highlightedComparisonText += f'<mark>{comparisonText[j1:j2]}</mark>'  
    return highlightedComparisonText

def cleanText(text):
    replacements = [
         # for the transcoded strings
         ('00_P', ''),  # Remove '00_P'
         ('00_S', ''),  # Remove '00_S'
         ('00', ''),    # Remove '00'
         ('&', ' '),    # Replace '&' with a space
         # for the Hebrew strings
         ('ס ', ''),    # Final Samekh
         ('פ ', ''),    # Final Pe
         ('׃', ''),     # End of verse
         ('־',' ')      # maqaf
    ]
    # Apply each replacement
    for old, new in replacements:
        text = text.replace(old, new)
    return text

# Function to format and highlight verse differences between MT and SP
def formatAndHighlight(label, MTverseText, SPverseText):
    book, chapter, verse = label
    MTverseNode = Tmt.nodeFromSection(label)
    MTtext = cleanText(Tmt.text(MTverseNode, "text-orig-plain"))
    SPverseNode = Tsp.nodeFromSection(label)
    SPtext = Tsp.text(SPverseNode)
    SPmarkedText = highlightMatches(MTtext, SPtext)
    MTmarkedText = highlightMatches(SPtext, MTtext)
    formattedDiff = (
        f'<h4>{book} {chapter}:{verse}</h4>'
        f'<p><b>SP:</b> {SPmarkedText}<br><b>MT:</b> {MTmarkedText}</p>'
    )
    return formattedDiff

# Create header
MT.dm(f'### Differences between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')

# Highlight and display differences for SP and MT verses
for label, MTverseText in MTverses.items():
    SPverseText = SPverses.get(label, '')
    MTverseText=cleanText(MTverseText)
    if MTverseText != SPverseText:  # Check for differences
        difference = formatAndHighlight(label, MTverseText, SPverseText)
        MT.dm(difference)

### Differences between MT and SP for parasha Noach (Genesis 6:9-11:32)

<h4>Genesis 6:9</h4><p><b>SP:</b> אלה תולדת נח נח איש<mark></mark> צדיק <mark>ו</mark>תמים היה בדרתיו את האלהים התהלך נח <br><b>MT:</b> אלה תולדת נח נח איש<mark>ׁ</mark> צדיק <mark></mark>תמים היה בדרתיו את האלהים התהלך נח </p>

<h4>Genesis 6:10</h4><p><b>SP:</b> ויול<mark>י</mark>ד נח ש<mark></mark>לש<mark></mark>ה בנים את ש<mark></mark>ם <mark>ו</mark>את חם ואת יפת <br><b>MT:</b> ויול<mark></mark>ד נח ש<mark>ׁ</mark>לש<mark>ׁ</mark>ה בנים את ש<mark>ׁ</mark>ם <mark></mark>את חם ואת יפת </p>

<h4>Genesis 6:13</h4><p><b>SP:</b> ויאמר אלהים לנח קץ כל בש<mark></mark>ר בא לפני כי מלאה הארץ חמ<mark>ס </mark>מפניהם והנני מש<mark></mark>חית<mark>י</mark>ם את הארץ <br><b>MT:</b> ויאמר אלהים לנח קץ כל בש<mark>ׂ</mark>ר בא לפני כי מלאה הארץ חמ<mark></mark>מפניהם והנני מש<mark>ׁ</mark>חית<mark></mark>ם את הארץ </p>

<h4>Genesis 6:15</h4><p><b>SP:</b> וזה אש<mark></mark>ר תעש<mark></mark>ה אתה ש<mark></mark>לש<mark></mark> מאות אמה ארך התבה <mark>ו</mark>חמש<mark></mark>ים אמה רחבה וש<mark></mark>לש<mark></mark>ים אמה קומתה <br><b>MT:</b> וזה אש<mark>ׁ</mark>ר תעש<mark>ׂ</mark>ה אתה ש<mark>ׁ</mark>לש<mark>ׁ</mark> מאות אמה ארך התבה <mark></mark>חמש<mark>ׁ</mark>ים אמה רחבה וש<mark>ׁ</mark>לש<mark>ׁ</mark>ים אמה קומתה </p>

...

<h4>Genesis 11:26</h4><p><b>SP:</b> ויחי תרח ש<mark></mark>בעים ש<mark></mark>נה ויול<mark>י</mark>ד את אברם <mark>ו</mark>את נחור ואת הרן <br><b>MT:</b> ויחי תרח ש<mark>ׁ</mark>בעים ש<mark>ׁ</mark>נה ויול<mark></mark>ד את אברם <mark></mark>את נחור ואת הרן </p>

<h4>Genesis 11:27</h4><p><b>SP:</b> <mark></mark>אלה תולדת תרח תרח הוליד את אברם <mark>ו</mark>את נחור ואת הרן והרן הוליד את לוט <br><b>MT:</b> <mark>ו</mark>אלה תולדת תרח תרח הוליד את אברם <mark></mark>את נחור ואת הרן והרן הוליד את לוט </p>

<h4>Genesis 11:30</h4><p><b>SP:</b> ותהי ש<mark></mark>רי עקרה אין לה <mark>י</mark>לד <br><b>MT:</b> ותהי ש<mark>ׂ</mark>רי עקרה אין לה <mark>ו</mark>לד </p>

<h4>Genesis 11:31</h4><p><b>SP:</b> ויקח תרח את אברם בנו ואת לוט בן הרן בן בנו ואת ש<mark></mark>רי <mark>ואת מלכה </mark>כל<mark>ו</mark>תו אש<mark></mark>ת אברם <mark>ונחור </mark>בנ<mark>י</mark>ו וי<mark>ו</mark>צא<mark></mark> אתם מאור כש<mark></mark>דים ללכת ארצה כנען ויבאו עד חרן ויש<mark></mark>בו ש<mark></mark>ם <br><b>MT:</b> ויקח תרח את אברם בנו ואת לוט בן הרן בן בנו ואת ש<mark>ׂ</mark>רי <mark></mark>כל<mark></mark>תו אש<mark>ׁ</mark>ת אברם <mark></mark>בנ<mark></mark>ו וי<mark></mark>צא<mark>ו</mark> אתם מאור כש<mark>ׂ</mark>דים ללכת ארצה כנען ויבאו עד חרן ויש<mark>ׁ</mark>בו ש<mark>ׁ</mark>ם </p>

<h4>Genesis 11:32</h4><p><b>SP:</b> ויהיו ימי תרח חמש<mark></mark> ש<mark></mark>נ<mark>ים וארבע</mark>ים ומאת<mark></mark> ש<mark></mark>נה וימת תרח בחרן <br><b>MT:</b> ויהיו ימי תרח חמש<mark>ׁ</mark> ש<mark>ׁ</mark>נ<mark></mark>ים ומאת<mark>ים</mark> ש<mark>ׁ</mark>נה וימת תרח בחרן </p>

# 4 - Compare texts using minimum Levenshtein distance<a class="anchor" id="bullet4"></a>
##### [Back to ToC](#TOC)

The Levenshtein distance measures the minimum number of single-character edits (insertions, deletions, or substitutions) needed to transform one text into another, providing a quantitative way to compare textual differences. For comparing the Masoretic Text and Samaritan Pentateuch, it highlights variations in spelling, word order, or minor textual changes. 
In the context of the Levenshtein distance (in the script below `threshold`), a higher number indicates greater dissimilarity between two texts, meaning more edits (insertions, deletions, or substitutions) are needed to transform one text into the other.

In [6]:
# Compare verses based on Levenshtein distance
from Levenshtein import distance
threshold = 20

# Create header
MT.dm(f'### Levenshtein distance {threshold} between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')

for label, MTverseText in MTverses.items():
    SPverseText = SPverses.get(label, '')
    if distance(MTverseText, SPverseText) > threshold:
        formattedDiff = formatAndHighlight(label, MTverseText, SPverseText)
        MT.dm(formattedDiff)

### Levenshtein distance 20 between MT and SP for parasha Noach (Genesis 6:9-11:32)

<h4>Genesis 6:20</h4><p><b>SP:</b> <mark>והיה </mark>מ<mark>ן </mark>העוף למינהו ומן הבהמה למינה <mark>ו</mark>מכל <mark>אשר </mark>רמש<mark> על</mark> האדמה למינ<mark>י</mark>ה<mark>ם</mark> ש<mark></mark>נים מכל יבאו אליך להחיות <br><b>MT:</b> <mark></mark>מ<mark></mark>העוף למינהו ומן הבהמה למינה <mark></mark>מכל <mark></mark>רמש<mark>ׂ</mark> האדמה למינ<mark></mark>ה<mark>ו</mark> ש<mark>ׁ</mark>נים מכל יבאו אליך להחיות </p>

<h4>Genesis 7:2</h4><p><b>SP:</b> מכל<mark></mark> הבהמה הטה<mark></mark>רה תקח לך ש<mark></mark>בעה ש<mark></mark>בעה <mark>זכר</mark> ו<mark>נקבה</mark> ומן הבהמה אש<mark></mark>ר לא טהרה ה<mark>י</mark>א ש<mark></mark>נים <mark>שנ</mark>י<mark>ם זכר</mark> ו<mark>נקבה</mark> <br><b>MT:</b> מכל<mark>׀</mark> הבהמה הטה<mark>ו</mark>רה תקח לך ש<mark>ׁ</mark>בעה ש<mark>ׁ</mark>בעה <mark>אישׁ</mark> ו<mark>אשׁתו</mark> ומן הבהמה אש<mark>ׁ</mark>ר לא טהרה ה<mark>ו</mark>א ש<mark>ׁ</mark>נים <mark>אי</mark>ש<mark>ׁ</mark> ו<mark>אשׁתו</mark> </p>

<h4>Genesis 8:19</h4><p><b>SP:</b> <mark>ו</mark>כל החיה<mark></mark> וכל העוף <mark>ו</mark>כל <mark>ה</mark>ר<mark></mark>מש<mark> הרמש</mark> על הארץ למש<mark></mark>פח<mark>ו</mark>תיהם יצאו מן התבה <br><b>MT:</b> <mark></mark>כל החיה<mark> כל הרמשׂ</mark> וכל העוף <mark></mark>כל <mark></mark>ר<mark>ו</mark>מש<mark>ׂ</mark> על הארץ למש<mark>ׁ</mark>פח<mark></mark>תיהם יצאו מן התבה </p>

<h4>Genesis 9:15</h4><p><b>SP:</b> וזכרתי את בריתי אש<mark></mark>ר ביני ובינ<mark></mark>כם ובין כל נפש<mark></mark> <mark>ה</mark>חיה<mark> אשר אתכם</mark> בכל בש<mark></mark>ר ולא יהיה עוד המים למבול ל<mark>ה</mark>ש<mark></mark>ח<mark>י</mark>ת כל בש<mark></mark>ר <br><b>MT:</b> וזכרתי את בריתי אש<mark>ׁ</mark>ר ביני ובינ<mark>י</mark>כם ובין כל נפש<mark>ׁ</mark> <mark></mark>חיה<mark></mark> בכל בש<mark>ׂ</mark>ר ולא יהיה עוד המים למבול ל<mark></mark>ש<mark>ׁ</mark>ח<mark></mark>ת כל בש<mark>ׂ</mark>ר </p>

...

<h4>Genesis 11:19</h4><p><b>SP:</b> ויחי פלג אחרי הולידו את רעו תש<mark></mark>ע ש<mark></mark>נים ומאת<mark></mark> ש<mark></mark>נה ויול<mark>י</mark>ד בנים ובנות <mark>ויהיו כל ימי פלג תשע ושלשים ומאתים שנה וימת </mark><br><b>MT:</b> ויחי פלג אחרי הולידו את רעו תש<mark>ׁ</mark>ע ש<mark>ׁ</mark>נים ומאת<mark>ים</mark> ש<mark>ׁ</mark>נה ויול<mark></mark>ד בנים ובנות <mark></mark></p>

<h4>Genesis 11:21</h4><p><b>SP:</b> ויחי רעו אחרי הולידו את ש<mark></mark>רוג ש<mark></mark>בע ש<mark></mark>נים ומאת<mark></mark> ש<mark></mark>נה ויול<mark>י</mark>ד בנים ובנות <mark>ויהיו כל ימי רעו תשע ושלשים ומאתים שנה וימת </mark><br><b>MT:</b> ויחי רעו אחרי הולידו את ש<mark>ׂ</mark>רוג ש<mark>ׁ</mark>בע ש<mark>ׁ</mark>נים ומאת<mark>ים</mark> ש<mark>ׁ</mark>נה ויול<mark></mark>ד בנים ובנות <mark></mark></p>

<h4>Genesis 11:23</h4><p><b>SP:</b> ויחי ש<mark></mark>רוג אחרי הולידו את נחור מאת<mark></mark> ש<mark></mark>נה ויול<mark>י</mark>ד בנים ובנות <mark>ויהיו כל ימי שרוג שלשים שנה ומאתים שנה וימת </mark><br><b>MT:</b> ויחי ש<mark>ׂ</mark>רוג אחרי הולידו את נחור מאת<mark>ים</mark> ש<mark>ׁ</mark>נה ויול<mark></mark>ד בנים ובנות <mark></mark></p>

<h4>Genesis 11:25</h4><p><b>SP:</b> ויחי נחור אחרי הולידו את תרח תש<mark></mark>ע <mark></mark>ש<mark>נים וששים</mark> ש<mark></mark>נה ויול<mark>י</mark>ד בנים ובנות <mark>ויהיו כל ימי נחור שמנה וארבעים שנה ומאת שנה וימת </mark><br><b>MT:</b> ויחי נחור אחרי הולידו את תרח תש<mark>ׁ</mark>ע <mark>ע</mark>ש<mark>ׂרה שׁ</mark>נ<mark>ה</mark> ו<mark>מאת</mark> ש<mark>ׁ</mark>נה ויול<mark></mark>ד בנים ובנות <mark></mark></p>

<h4>Genesis 11:31</h4><p><b>SP:</b> ויקח תרח את אברם בנו ואת לוט בן הרן בן בנו ואת ש<mark></mark>רי <mark>ואת מלכה </mark>כל<mark>ו</mark>תו אש<mark></mark>ת אברם <mark>ונחור </mark>בנ<mark>י</mark>ו וי<mark>ו</mark>צא<mark></mark> אתם מאור כש<mark></mark>דים ללכת ארצה כנען ויבאו עד חרן ויש<mark></mark>בו ש<mark></mark>ם <br><b>MT:</b> ויקח תרח את אברם בנו ואת לוט בן הרן בן בנו ואת ש<mark>ׂ</mark>רי <mark></mark>כל<mark></mark>תו אש<mark>ׁ</mark>ת אברם <mark></mark>בנ<mark></mark>ו וי<mark></mark>צא<mark>ו</mark> אתם מאור כש<mark>ׂ</mark>דים ללכת ארצה כנען ויבאו עד חרן ויש<mark>ׁ</mark>בו ש<mark>ׁ</mark>ם </p>

# 5 - Comparison of spelling of proper nouns between SP and MT<a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

This section focuses on comparing the spelling of proper nouns between the Samaritan Pentateuch (SP) and the Masoretic Text (MT). Proper nouns, including names of people, places, and unique terms, often exhibit variations in spelling

In [7]:
import collections

def collectProperNounSpellings(F, L, T, inputList):
    """
    Collect proper noun spellings and their associated word node numbers.
    Ensures only one tuple is stored for each lexeme-to-spelling mapping.
    """
    properNounsSpellings = {}
    for bookChapterVerse in inputList:
        verseNode = T.nodeFromSection(bookChapterVerse)
        wordNodes = L.d(verseNode, 'word')
        for wordNode in wordNodes:
            if F.sp.v(wordNode) == 'nmpr':  # Check if the word is a proper noun
                lex = F.lex.v(wordNode)    # Lexical form
                spelling = F.g_cons.v(wordNode)  # Spelling
                # Store only the first occurrence for each lex-to-cons mapping
                if lex not in properNounsSpellings or spelling not in {item[0] for item in properNounsSpellings[lex]}:
                    properNounsSpellings.setdefault(lex, []).append((spelling, wordNode))
    return properNounsSpellings
        
SPspellingDict = collectProperNounSpellings(Fsp, Lsp, Tsp, bookChapterVerseList) 
MTspellingDict = collectProperNounSpellings(Fmt, Lmt, Tmt, bookChapterVerseList)

In [8]:
# Create header
MT.dm(f'### Spelling differences proper nouns in SP and MT for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')
    
for lex, MTspellings in MTspellingDict.items():
    # Retrieve SP spellings, defaulting to an empty set if lex is not found
    SPspellings = SPspellingDict.get(lex, set())

    # Extract only the spellings (ignoring node numbers) for comparison
    MTspellingSet = {spelling for spelling, _ in MTspellings}
    SPspellingSet = {spelling for spelling, _ in SPspellings}

    # Compare the sets of spellings
    if MTspellingSet != SPspellingSet:
        # Print MT spelling with reference
        MTnode = list(MTspellings)[0][1]  # Get first tuple's node number
        book, chapter, verse = Tmt.sectionFromNode(MTnode)
        MTgloss = Fmt.gloss.v(MTnode)
        MTspelling = Fmt.g_cons_utf8.v(MTnode)

        # Build Markdown output
        output = f"#### Word: **{MTgloss}** ({book} {chapter}:{verse})\n"
        output += f"- **MT Spelling:** `{MTspelling}`\n"

        # Print SP spellings with reference
        if SPspellings:
            SPnode = list(SPspellings)[0][1]  # Get first tuple's node number
            SPspelling = Fsp.g_cons_utf8.v(SPnode)
            output += f"- **SP Spelling:** `{SPspelling}`\n"
        else:
            output += "- **SP Spelling:** None\n"

        # Print the formatted output
        MT.dm(output)

### Spelling differences proper nouns in SP and MT for parasha Noach (Genesis 6:9-11:32)

#### Word: **Ararat** (Genesis 8:4)
- **MT Spelling:** `אררט`
- **SP Spelling:** `הררט`


#### Word: **Tubal** (Genesis 10:2)
- **MT Spelling:** `תבל`
- **SP Spelling:** `תובל`


#### Word: **Meshech** (Genesis 10:2)
- **MT Spelling:** `משׁך`
- **SP Spelling:** `מושׁך`


#### Word: **Elishah** (Genesis 10:4)
- **MT Spelling:** `אלישׁה`
- **SP Spelling:** `אלישׁ`


...

#### Word: **Meshech** (Genesis 10:23)
- **MT Spelling:** `משׁ`
- **SP Spelling:** `משׁא`


#### Word: **Hadoram** (Genesis 10:27)
- **MT Spelling:** `הדורם`
- **SP Spelling:** `אדורם`


#### Word: **Uzal** (Genesis 10:27)
- **MT Spelling:** `אוזל`
- **SP Spelling:** `עיזל`


#### Word: **Obal** (Genesis 10:28)
- **MT Spelling:** `עובל`
- **SP Spelling:** `עיבל`


#### Word: **<land of gold>** (Genesis 10:29)
- **MT Spelling:** `אופר`
- **SP Spelling:** `אפר`


# 6 - References and acknowledgement <a class="anchor" id="bullet6"></a>
##### [Back to ToC](#TOC)

<a class="anchor" id="bullet1"><sup>1</sup></a> Christian Canu Højgaard, Martijn Naaijer, & Stefan Schorch. (2023). Text-Fabric Dataset of the Samaritan Pentateuch. Zenodo. https://doi.org/10.5281/zenodo.7734632

<a class="anchor" id="bullet2"><sup>2</sup> [Notebook created by Martijn Naaijer](https://github.com/DT-UCPH/sp/blob/main/notebooks/combine_sp_with_mt_data.ipynb)

# 7 - Required libraries <a class="anchor" id="bullet7"></a>
##### [Back to ToC](#TOC)

The scripts in this notebook require (beside `text-fabric`) the following Python libraries to be installed in the environment:

    collections
    difflib
    Levenshtein

You can install any missing library from within Jupyter Notebook using either`pip` or `pip3`.

# 8 - Notebook version details<a class="anchor" id="bullet8"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.0</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>17 November 2024</td>
    </tr>
  </table>
</div>